In [ ]:
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
data=pd.read_csv("AAG89Dit.csv")
target_label = 'POLAFF'
data.shape

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (13,15,16,18,32,34,36,37,41,45,46,55,71,83,84,86,87,90,91,92,96,98,100,101,103,104,105,106,107,108,109) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


(309119, 111)

In [ ]:
empty = []
for i in range(data.shape[0]):
  if str(data[target_label][i]).isspace() or float(data[target_label][i])==8. or float(data[target_label][i])==9. or float(data[target_label][i])==2.:
      empty.append(i)
# empty

In [ ]:
data=data.drop(empty).reset_index(drop=True)
data[target_label]

0        3.0
1        3.0
2        3.0
3        3.0
4        3.0
        ... 
42170      3
42171      3
42172      3
42173      3
42174      1
Name: POLAFF, Length: 42175, dtype: object

In [ ]:
data

,CASEID,YYYYMM,YYYYQ,YYYY,ID,IDPREV,DATEPR,ICS,ICC,ICE,...,GAS1PX1,GAS1PX2,GAS1,PINC,PINC2,PJOB,PSSA,PCRY,PSTK,WT
0,28064,198006,19802,1980,1,,,2.70,2.70,2.70,...,,,,,,,,,,2.65
1,28065,198006,19802,1980,2,,,91.51,78.39,99.94,...,,,,,,,,,,1.77
2,28067,198006,19802,1980,4,,,47.11,78.39,27.01,...,,,,,,,,,,1.32
3,28068,198006,19802,1980,5,,,17.50,40.54,2.70,...,,,,,,,,,,0.00
4,28069,198006,19802,1980,6,,,47.11,78.39,27.01,...,,,,,,,,,,0.88
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61965,309115,202301,20231,2023,2096,1049,202207,76.01,77.69,74.93,...,1,25,25,80.0,0.0,20.0,10.0,1,90.0,1.00
61966,309116,202301,20231,2023,2097,1144,202207,120.42,153.38,99.24,...,3,,0,95.0,95.0,20.0,80.0,3,60.0,1.25
61967,309117,202301,20231,2023,2098,1096,202207,31.60,39.84,26.31,...,3,,0,0.0,70.0,0.0,100.0,3,50.0,0.50
61968,309118,202301,20231,2023,2099,1121,202207,46.41,77.69,26.31,...,3,,0,50.0,100.0,5.0,60.0,3,50.0,1.00


In [ ]:
for column in data.columns:
  for i in range(data.shape[0]):
    if str(data[column][i]).isspace():
        data[column][i] = 0

In [ ]:
for column in data.columns:
  for i in range(data.shape[0]):
    if type(data[column][i])==str:
      data[column][i] = float(data[column][i])

In [ ]:
import os
import torch
from torch import nn

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(110, 25),
            nn.ReLU(),
            nn.Linear(25, 5),
            nn.ReLU(),
            nn.Linear(5, 2),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [ ]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=110, out_features=25, bias=True)
    (1): ReLU()
    (2): Linear(in_features=25, out_features=5, bias=True)
    (3): ReLU()
    (4): Linear(in_features=5, out_features=2, bias=True)
  )
)


In [ ]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=110, out_features=25, bias=True)
    (1): ReLU()
    (2): Linear(in_features=25, out_features=5, bias=True)
    (3): ReLU()
    (4): Linear(in_features=5, out_features=2, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([25, 110]) | Values : tensor([[-3.8040e-03,  5.8287e-02,  3.1391e-02,  4.7080e-03, -7.1674e-02,
          8.8282e-02,  8.4743e-02, -5.2455e-05, -3.9049e-02, -8.1863e-02,
         -3.5134e-02,  6.0836e-02, -2.7474e-02,  5.6232e-02,  1.3672e-03,
         -6.9897e-03, -2.1360e-02, -5.9138e-02,  4.3404e-02,  2.2178e-03,
         -7.0990e-02,  9.3568e-02, -5.7862e-02, -6.5968e-02,  6.6091e-02,
         -4.6834e-02,  9.2972e-04,  2.8208e-02,  6.4699e-02,  6.0635e-02,
          1.0009e-02, -7.2941e-02, -6.0110e-03,  2.5828e-02, -9.1942e-02,
          7.6740e-02,  1.0081e-03,  3.6895e-02,  1.9192e-03, -1.9255e-02,
  

In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import os
from torchvision.transforms import ToTensor, Lambda

training_data = data[0:40000:]
testing_data = data[40000:]

training_data_results = training_data[target_label]
# for i in range(30000):
#   nlist = [0,0,0]
#   nlist[training_data_results[i]-1] = 1
testing_data_results = testing_data[target_label]
training_data = training_data.drop([target_label,], axis=1)
testing_data = testing_data.drop([target_label,], axis=1)   #.to_numpy()


class CustomImageDataset(Dataset):
    def __init__(self, data, label, target_transform=None):
        self.labels = label
        self.data = data
        self.target_transform = target_transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        itlabel = self.labels.iloc[idx]-1
        if itlabel==2:
          itlabel=1
        itdata = self.data.iloc[:,idx]
        itdata = torch.tensor(itdata)
        if self.target_transform:
          itlabel = self.target_transform(np.int64(itlabel))
        return itdata, itlabel

# actual_training_data = [(training_data[i:i+1,].T, training_data_results.iloc[i]) for i in range(training_data.shape[0])]
# actual_testing_data = [(testing_data[i:i+1,].T, testing_data_results.iloc[i]) for i in range(testing_data.shape[0])]

target_transform = Lambda(lambda y: torch.zeros(
    2, dtype=torch.float).scatter_(dim=0, index=torch.tensor(y), value=1))

actual_training_data=CustomImageDataset(training_data.T,training_data_results,target_transform)   # one-hot encoding
actual_testing_data=CustomImageDataset(testing_data.T,testing_data_results)

learning_rate = 1e-4
batch_size = 64
epochs = 5

train_dataloader = DataLoader(actual_training_data, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(actual_testing_data, batch_size=batch_size, shuffle=True)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 242865.062500  [   64/40000]
loss: 0.774912  [ 6464/40000]
loss: 0.700327  [12864/40000]
loss: 0.714860  [19264/40000]
loss: 0.726518  [25664/40000]
loss: 0.689062  [32064/40000]
loss: 0.698409  [38464/40000]
Test Error: 
 Accuracy: 40.6%, Avg loss: 0.705017 

Epoch 2
-------------------------------
loss: 0.706709  [   64/40000]
loss: 0.699678  [ 6464/40000]
loss: 0.702723  [12864/40000]
loss: 0.695914  [19264/40000]
loss: 0.697385  [25664/40000]
loss: 0.695228  [32064/40000]
loss: 0.693248  [38464/40000]
Test Error: 
 Accuracy: 59.4%, Avg loss: 0.692857 

Epoch 3
-------------------------------
loss: 0.692809  [   64/40000]
loss: 0.690087  [ 6464/40000]
loss: 0.692772  [12864/40000]
loss: 0.689058  [19264/40000]
loss: 0.687965  [25664/40000]
loss: 0.685920  [32064/40000]
loss: 0.681187  [38464/40000]
Test Error: 
 Accuracy: 59.4%, Avg loss: 0.685995 

Epoch 4
-------------------------------
loss: 0.696755  [   64/40000]
loss: 0.688322  [ 6

In [ ]:
X = torch.rand(64, 110, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1,
        0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1,
        0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1])


In [ ]:
count = 0

for i in range(len(testing_data)):
  if testing_data_results.iloc[i]==3:
    count+=1

count/len(testing_data)

0.5940229885057471

LMAO, although the model doesn't seem to only produce democrats, it seems all the predictions made in the testing data was democrats and thus resulting in a 59.4% accuracy.

In [ ]:
torch.save(model, 'model.pth')

In [ ]:
# model = torch.load('model.pth')